In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

16711680/16705208 [==============================] - 2s 0us/step


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 28s - loss: 0.8586 - auc: 0.4985 - accuracy: 0.4990 - val_loss: 0.7554 - val_auc: 0.5000 - val_accuracy: 0.1719
Epoch 2/30
40/40 - 28s - loss: 0.7114 - auc: 0.5107 - accuracy: 0.5140 - val_loss: 0.6935 - val_auc: 0.5000 - val_accuracy: 0.3151
Epoch 3/30
40/40 - 27s - loss: 0.7199 - auc: 0.5032 - accuracy: 0.5010 - val_loss: 0.7115 - val_auc: 0.5016 - val_accuracy: 0.1719
Epoch 4/30
40/40 - 26s - loss: 0.7247 - auc: 0.4785 - accuracy: 0.4817 - val_loss: 0.7941 - val_auc: 0.5000 - val_accuracy: 0.1719
Epoch 5/30
40/40 - 26s - loss: 0.7596 - auc: 0.4847 - accuracy: 0.4904 - val_loss: 0.7526 - val_auc: 0.5720 - val_accuracy: 0.1719
Epoch 6/30
40/40 - 26s - loss: 0.7069 - auc: 0.5062 - accuracy: 0.5002 - val_loss: 0.6066 - val_auc: 0.5023 - val_accuracy: 0.8281
Epoch 7/30
40/40 - 26s - loss: 0.7540 - auc: 0.5028 - accuracy: 0.5033 - val_loss: 0.9937 - val_auc: 0.5000 - val_accuracy: 0.1693
Epoch 8/30
40/40 - 27s - loss: 0.7660 - auc: 0.5250 - accuracy: 0.5140 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

0
386


In [8]:
prediction

array([[0.45801795],
       [0.45783508],
       [0.45660728],
       [0.4584942 ],
       [0.4574837 ],
       [0.4567834 ],
       [0.45652443],
       [0.45719567],
       [0.45762268],
       [0.4573227 ],
       [0.45740777],
       [0.45717102],
       [0.45791695],
       [0.45830563],
       [0.457765  ],
       [0.45634815],
       [0.4578807 ],
       [0.45812082],
       [0.45735794],
       [0.45679873],
       [0.456853  ],
       [0.45709723],
       [0.45673785],
       [0.45790994],
       [0.45799986],
       [0.45814484],
       [0.45744553],
       [0.4573103 ],
       [0.45722798],
       [0.45639625],
       [0.457726  ],
       [0.45671695],
       [0.45751935],
       [0.45758992],
       [0.45622763],
       [0.4571734 ],
       [0.45733106],
       [0.45741445],
       [0.4578807 ],
       [0.4570224 ],
       [0.4583193 ],
       [0.45793855],
       [0.45820057],
       [0.45729947],
       [0.4574492 ],
       [0.45727623],
       [0.4570421 ],
       [0.456

In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

7/7 [==============================] - 3s 459ms/step - loss: 0.7528 - auc: 0.5716 - accuracy: 0.1710


[0.7527821660041809, 0.5716382265090942, 0.17098446190357208]